In [ ]:
import os
env = {
    'AWS_REGION': 'us-standard',
    'S3_ENDPOINT': 's3.us.cloud-object-storage.appdomain.cloud',
    'AWS_ACCESS_KEY_ID': 'dcd241d33c7142d6b8a80251e05375fc',
    'AWS_SECRET_ACCESS_KEY': 'd30074e169ee8aa885dba41819c247118e163878f47abc32',
    'S3_USE_HTTPS': '1',
    'S3_VERIFY_SSL': '0',
}
os.environ.update(env)

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import sys
sys.path.append('../gpu-src/')
sys.path.append('../lottery/')
import official.resnet.cifar10_main
import matplotlib.pyplot as plt

In [ ]:
prune_idxs = sorted(list(map(int, '12514 14 1576 16420 20326 24233 28139 32045 35951 39858 404 43764 4701 47670 51576 55483 59389 63295 67201 69545 70326 70717 71108 795 8608'.split())))
prune_idxs = [0,4701,12514,20326,28139,35951,39858,43764,51576,59389,67201,71108]

In [ ]:
tf.logging.set_verbosity(tf.logging.WARN)

In [ ]:
def get_est(prune_idx, trial, iter_idx):
    est = tf.estimator.Estimator(
        official.resnet.cifar10_main.cifar10_model_fn,
        's3://REDACTED-data/execution_data/resnet20/prune_resnet20_structured_discovered/v5/lottery/prune_{}/trial_{}/iter_{}'.format(
#         'gs://REDACTED/execution_data/resnet20/global_magnitude_20/v2/lottery/prune_{}/trial_{}/iter_{}'.format(
            prune_idx,
            trial,
            iter_idx
        ),
        params={
          'resnet_size': 20,
          'data_format': 'channels_first',
          'batch_size': 128,
          'resnet_version': 1,
          'loss_scale': 1,
          'dtype': tf.float32,
          'fine_tune': False
      }
    )
    try:
        est.get_variable_names()
    except ValueError:
        raise
        return None
    return est

In [ ]:
def dists(est_a, est_b):
    names = est_a.get_variable_names()
    ws = [x for x in names if x.endswith('_w')]
    ms = [x for x in names if x.endswith('_m')]
    
    d = 0
    n = 0
    for w,m in list(zip(ws, ms)):
        a_w = est_a.get_variable_value(w)
        a_m = est_a.get_variable_value(m)
        b_w = est_b.get_variable_value(w)
        b_m = est_b.get_variable_value(m)
        
        d += np.linalg.norm(a_w*a_m*b_m - b_w*a_m*b_m)
        n += (a_m*b_m).sum()
    return d / n

In [ ]:
def get_trial_stability(prune_idx, trial_idx, iter_idx):
    a = get_est(prune_idx, trial_idx, 0)
    b = get_est(prune_idx, trial_idx, iter_idx)
    if a is None or b is None:
        return None
    return dists(a, b)

def get_all_stabilities(prune_idx, iter_idx):
    res = []
    p = multiprocessing.Pool(3)
    res = np.array(
        list(filter(lambda x: x is not None, 
                    p.starmap(get_trial_stability, [(prune_idx, i, iter_idx) for i in range(1,4)])))
    )
    p.close()
    return res

In [ ]:
N_WORKERS = 23

import queue
import threading
import multiprocessing.dummy

all_results = []

for res_idx in range(1, 8):
    
    q = queue.Queue()
    for p in prune_idxs:
        q.put(p)

    for _ in range(N_WORKERS):
        q.put(None)
    
    def get_stab(idx):
        return get_all_stabilities(idx, res_idx)
    
    results = {}
    all_results.append(results)

    def i_hate_tensorflow():
        while True:
            pidx = q.get()
            if pidx is None:
                 break
            print(pidx)
            p = multiprocessing.dummy.Pool(1)
            results[pidx] = p.apply(get_stab, [pidx])
            p.close()

    tds = []
    for _ in range(N_WORKERS):
        tds.append(threading.Thread(target=i_hate_tensorflow))
        tds[-1].start()

    for t in tds:
        t.join()

In [ ]:
# plt.figure(figsize=(8 ,32))
for i, results in enumerate(all_results):
#     plt.subplot(len(accs), 1, i+1)
    plt.title('Iteration {}'.format(i+1))
    plt.figure(figsize=(8 ,4))
    x, y = zip(*sorted(results.items()))
    plt.errorbar(
        np.array(x) / (50000/128),
        np.array([np.median(s) for s in y]),
        yerr=np.array([[s.mean() - s.min() for s in y], [s.max() - s.mean() for s in y]]),
        fmt='o--',
        label='Stability'
    )
    plt.legend(loc=3)
    
#     ax = plt.twinx()
#     lot_x, lot_accs = zip(*acc)
#     ax.errorbar(
#         np.array(lot_x) ,
#         [np.array(y).mean() for y in lot_accs],
#         yerr=[np.array(y).std() for y in lot_accs],
#         fmt='o--',
#         color='C1',
#         label=r'$\Delta$ Accuracy'
#     )
#     ax.set_ylim(-.01, .01)
#     ax.legend(loc=1)

    # plt.xlim(0, 10000)
plt.tight_layout()
plt.show()

In [ ]:
from numpy import array
accs = [[(0,
   array([ 0.00059998, -0.0018    , -0.00010002, -0.00150001, -0.00139999])),
  (1, array([0.00020003, 0.00059998, 0.00349998, 0.00150001])),
  (2,
   array([-0.00010002, -0.00089997,  0.0011    , -0.00169998, -0.00029999])),
  (4,
   array([ 0.00120002,  0.00040001,  0.00099999,  0.00050002, -0.00080001])),
  (10,
   array([ 0.00019997, -0.00099999, -0.00059998, -0.00019997, -0.00059998])),
  (20, array([-0.0007    , -0.00139999, -0.00129998,  0.00140005])),
  (30,
   array([ 0.00010002, -0.00049996, -0.0011    , -0.0018    ,  0.00150001])),
  (40,
   array([-0.00040001, -0.00029999,  0.00019997, -0.00020003,  0.00059998])),
  (50,
   array([-0.00049996,  0.        , -0.00259995, -0.00140005, -0.00139999])),
  (60,
   array([ 9.99569893e-05, -8.99970531e-04,  1.10000372e-03,  4.00006771e-04,
          -8.99970531e-04])),
  (70,
   array([ 0.00090003,  0.00139999,  0.00189996, -0.00210005,  0.00059998])),
  (80,
   array([-5.00023365e-04,  1.20002031e-03, -5.99980354e-04,  4.00006771e-04,
           9.99569893e-05])),
  (90,
   array([-0.00100005,  0.00060004, -0.00269997,  0.0011    ,  0.00139999])),
  (100,
   array([-0.0043    , -0.00569999, -0.00340003, -0.00029999,  0.00080001])),
  (110,
   array([ 0.00209999, -0.00150001,  0.00300002,  0.00350004, -0.00029999])),
  (120,
   array([-0.00120002, -0.0011    ,  0.0025    , -0.0025    , -0.00190002])),
  (130,
   array([-0.00080001,  0.00330001,  0.0011    , -0.00010002,  0.0007    ])),
  (140,
   array([-0.00059998,  0.00089997,  0.00130004,  0.00029999, -0.00020003])),
  (150,
   array([ 1.00004673e-03,  6.99996948e-04, -9.99569893e-05,  3.30001116e-03,
           4.99999523e-03])),
  (160,
   array([-0.00040001, -0.00319999,  0.00620002, -0.00190002, -0.00199997])),
  (170,
   array([ 0.00400001,  0.00380003,  0.00050002, -0.00410002, -0.00129998])),
  (178,
   array([-0.00060004,  0.00239998,  0.00220001,  0.0068    ,  0.00059998])),
  (180,
   array([ 0.0011    , -0.00560004, -0.00040001, -0.0025    ,  0.00150001])),
  (181,
   array([ 0.00059998, -0.00209999, -0.00339997, -0.0061    , -0.00220001])),
  (182,
   array([-4.10002470e-03,  9.99569893e-05, -1.90001726e-03, -1.99997425e-03,
          -4.40001488e-03]))],
 [(0,
   array([-0.00650001, -0.0061    , -0.00639999, -0.00419998, -0.00819999])),
  (1, array([-0.0036    , -0.00120002,  0.00080001,  0.00099999])),
  (2,
   array([-0.00120002, -0.00040001,  0.00059998, -0.00190002, -0.00199997])),
  (4,
   array([ 0.0007    ,  0.00130004,  0.00159997, -0.00159997, -0.00050002])),
  (10,
   array([ 0.        , -0.00050002, -0.00260001, -0.00150001, -0.00229996])),
  (20, array([ 0.00010002,  0.00050002, -0.00049996,  0.00060004])),
  (30,
   array([-0.00059998, -0.00049996, -0.00269997,  0.00080001,  0.00129998])),
  (40,
   array([-0.00080001, -0.0007    ,  0.00029999, -0.0007    ,  0.00040001])),
  (50,
   array([ 0.00040001, -0.00010002, -0.00369996, -0.00010002, -0.0007    ])),
  (60,
   array([ 0.00129998,  0.00090003,  0.00209999, -0.00159997, -0.0011    ])),
  (70,
   array([-0.00049996,  0.0011    ,  0.0029    , -0.00130004,  0.00029999])),
  (80,
   array([-0.00220001,  0.00169998, -0.00449997,  0.00029999,  0.00229996])),
  (90,
   array([ 0.0025    ,  0.00090003, -0.00190002, -0.00169998,  0.00340003])),
  (100,
   array([-0.0018    , -0.00130004, -0.00150001, -0.00040001,  0.00340003])),
  (110,
   array([ 0.0036    , -0.00019997,  0.00389999,  0.0029    , -0.0043    ])),
  (120,
   array([ 0.00259995, -0.00080001,  0.00050002,  0.00040001,  0.00169998])),
  (130,
   array([-0.00050002,  0.00220001,  0.00730002,  0.00099999,  0.00219995])),
  (140,
   array([-0.00189996,  0.00319999,  0.0018    ,  0.00580001,  0.00729996])),
  (150, array([0.00260001, 0.00129998, 0.00010002, 0.00059998, 0.0029    ])),
  (160,
   array([ 0.00239998,  0.00049996,  0.00630003, -0.00220001,  0.00090003])),
  (170, array([0.0036    , 0.00340003, 0.00370002, 0.00079995, 0.00210005])),
  (178, array([0.00059998, 0.00529999, 0.00379997, 0.00440001, 0.00099999])),
  (180,
   array([ 0.00190002, -0.00090003,  0.00419998,  0.00130004,  0.00389999])),
  (181,
   array([-0.00130004, -0.00099999, -0.01010001, -0.0036    , -0.00730002])),
  (182,
   array([-0.0072    , -0.005     , -0.00960004, -0.01109999, -0.00160003]))],
 [(0,
   array([-0.0499    , -0.0194    , -0.03430003, -0.02380002, -0.042     ])),
  (1, array([-0.00549996, -0.00050002, -0.00490004, -0.00650001])),
  (2,
   array([-0.00459999, -0.0029    , -0.00419998, -0.00560004, -0.00309998])),
  (4,
   array([-0.00590003, -0.00329995,  0.00040001, -0.00389999, -0.00310004])),
  (10,
   array([ 0.00019997, -0.00089997, -0.00370002, -0.00239998, -0.00369996])),
  (20, array([-0.00119996, -0.00169998, -0.00339997, -0.00099999])),
  (30,
   array([-0.00230002, -0.00080001, -0.00459999, -0.00209999, -0.00059998])),
  (40,
   array([-0.00130004, -0.00410002, -0.00020003, -0.00120002, -0.00150001])),
  (50,
   array([ 0.0025    , -0.00200003, -0.0018    , -0.00270003, -0.00239998])),
  (60,
   array([ 0.00059998, -0.00059998, -0.00080001, -0.00319999,  0.00120002])),
  (70,
   array([ 0.00060004, -0.00199997, -0.00020003,  0.00169998,  0.00029999])),
  (80,
   array([-0.00239998,  0.00090003, -0.00189996, -0.0018    , -0.0018    ])),
  (90,
   array([-0.00100005,  0.00050002, -0.00229996, -0.00079995,  0.00190002])),
  (100,
   array([-0.00260001, -0.00150001, -0.00190002, -0.00099999,  0.00459999])),
  (110,
   array([ 0.00669998, -0.00309998,  0.00050002,  0.00370002,  0.00029999])),
  (120,
   array([ 0.00279999, -0.0007    ,  0.0018    , -0.0036    , -0.0018    ])),
  (130,
   array([-0.00029999,  0.00410002,  0.00330001,  0.00349998,  0.00349998])),
  (140,
   array([-0.00169998,  0.00239998,  0.00330001,  0.005     ,  0.00599998])),
  (150,
   array([ 5.40000200e-03,  2.79998779e-03, -9.99569893e-05,  2.79998779e-03,
           3.89999151e-03])),
  (160,
   array([-0.00010002,  0.00150001,  0.00650001,  0.00200003,  0.00319999])),
  (170, array([0.00250006, 0.00440001, 0.00280005, 0.00059998, 0.00150001])),
  (178, array([0.00229996, 0.00760001, 0.00510001, 0.00589997, 0.00279999])),
  (180, array([0.        , 0.00089997, 0.00459999, 0.0025    , 0.0029    ])),
  (181,
   array([ 0.00019997, -0.0054    , -0.0086    , -0.00709999, -0.00669998])),
  (182,
   array([-0.00490004, -0.00750005, -0.00700003, -0.01099998, -0.00510001]))],
 [(0,
   array([-0.10970002, -0.05299997, -0.08230001, -0.04820001, -0.10289997])),
  (1, array([-0.01349998, -0.00810003, -0.01140004, -0.0151    ])),
  (2,
   array([-0.00949997, -0.00690001, -0.00749999, -0.0086    , -0.00919998])),
  (4,
   array([-0.00590003, -0.00659996, -0.0029    , -0.00800002, -0.00440001])),
  (10,
   array([-0.00230002, -0.00190002, -0.00580001, -0.00309998, -0.00580001])),
  (20, array([-0.00219995, -0.00139999, -0.00599998, -0.00419998])),
  (30,
   array([-0.00319999, -0.00089997, -0.00549996, -0.00450003, -0.00199997])),
  (40,
   array([-0.0018    , -0.00650001, -0.00300002, -0.0011    , -0.00420004])),
  (50,
   array([ 0.00050002, -0.00150001, -0.0036    , -0.00240004, -0.00139999])),
  (60,
   array([-0.00170004,  0.00170004, -0.00059998, -0.00289994, -0.0007    ])),
  (70,
   array([-0.00089997, -0.0018    ,  0.00059998,  0.00029999, -0.00190002])),
  (80,
   array([-0.0018    , -0.00150001, -0.00409997,  0.00059998,  0.00159997])),
  (90,
   array([-0.0029    , -0.00089997, -0.00309998, -0.00209999,  0.0007    ])),
  (100,
   array([-0.0007    , -0.00139999, -0.0029    , -0.00269997,  0.00389999])),
  (110,
   array([ 3.70001793e-03, -9.99569893e-05,  2.40004063e-03,  4.10002470e-03,
           0.00000000e+00])),
  (120,
   array([ 0.00199997, -0.00140005,  0.00390005, -0.00300002,  0.00019997])),
  (130,
   array([-0.0011    ,  0.00240004,  0.00260001,  0.00409997,  0.00619996])),
  (140,
   array([ 4.00006771e-04,  4.59998846e-03, -9.99569893e-05,  4.09996510e-03,
           4.49997187e-03])),
  (150, array([0.00740004, 0.00059998, 0.00350004, 0.0025    , 0.00779998])),
  (160,
   array([ 0.00330001,  0.00139999,  0.00800002, -0.0018    ,  0.00300002])),
  (170,
   array([ 0.00140005,  0.00550002,  0.0025    , -0.00080001,  0.00190002])),
  (178, array([0.00149995, 0.00730002, 0.00370002, 0.00730002, 0.00239998])),
  (180,
   array([ 0.00059998, -0.00080001,  0.0072    , -0.00229996,  0.0068    ])),
  (181,
   array([-0.00220001, -0.00980002, -0.00999999, -0.00819999, -0.0072    ])),
  (182,
   array([-0.01250005, -0.0072    , -0.01000005, -0.0176    , -0.01270002]))],
 [(0,
   array([-0.27950001, -0.15719998, -0.19770002, -0.1455    , -0.27020001])),
  (1, array([-0.02089995, -0.0187    , -0.02759999, -0.04159999])),
  (2,
   array([-0.01359999, -0.0151    , -0.01539999, -0.01719999, -0.01159996])),
  (4,
   array([-0.01370001, -0.01189995, -0.00810003, -0.01069999, -0.00670004])),
  (10,
   array([-0.0068    , -0.00690001, -0.0072    , -0.00599998, -0.01010001])),
  (20, array([-0.00589997, -0.00419998, -0.00639999, -0.00529999])),
  (30,
   array([-0.00620002, -0.00209999, -0.00749999, -0.0061    , -0.00529999])),
  (40,
   array([-0.00520003, -0.00599998, -0.00250006, -0.00590003, -0.00560004])),
  (50,
   array([-0.00139999, -0.00040001, -0.00699997, -0.00590003, -0.00349998])),
  (60,
   array([-0.00320005, -0.00229996, -0.00400001, -0.00399995, -0.0029    ])),
  (70,
   array([-0.00459999, -0.00119996, -0.00260001, -0.00080001, -0.00190002])),
  (80,
   array([-0.00150001, -0.00159997, -0.00279999, -0.00209999, -0.00320005])),
  (90,
   array([-0.00300002,  0.00050002, -0.00319999, -0.00349998,  0.00029999])),
  (100,
   array([-0.00300002, -0.00650001, -0.00310004, -0.00389999,  0.00400001])),
  (110,
   array([ 0.00379997,  0.00080001,  0.00240004,  0.00480002, -0.00120002])),
  (120,
   array([-0.00010002, -0.0007    ,  0.0011    , -0.00220001, -0.00310004])),
  (130,
   array([-0.00190002,  0.00190002,  0.00270003,  0.00330001,  0.00529999])),
  (140,
   array([-0.00409997,  0.00459999,  0.00030005,  0.0036    ,  0.00589997])),
  (150, array([0.00490004, 0.00099999, 0.00200003, 0.00129998, 0.00480002])),
  (160,
   array([-0.00099999,  0.00029999,  0.00100005,  0.00050002,  0.00029999])),
  (170, array([0.00320005, 0.009     , 0.00660002, 0.00059998, 0.00660002])),
  (178,
   array([-0.00280005,  0.0068    ,  0.00700003,  0.00620002,  0.0036    ])),
  (180,
   array([ 0.00049996,  0.00259995,  0.00440001, -0.00150001,  0.00260001])),
  (181,
   array([-0.0029    , -0.0104    , -0.01309997, -0.00529999, -0.0086    ])),
  (182,
   array([-0.01730001, -0.01560003, -0.01710004, -0.01899999, -0.01460004]))],
 [(0,
   array([-0.56430003, -0.38120002, -0.42430001, -0.3908    , -0.39059997])),
  (1, array([-0.051     , -0.04070002, -0.06200004, -0.07700002])),
  (2,
   array([-0.02359998, -0.0266    , -0.02390003, -0.02499998, -0.0201    ])),
  (4,
   array([-0.01899999, -0.01749998, -0.01209998, -0.0183    , -0.01429999])),
  (10,
   array([-0.00850004, -0.01359999, -0.01099998, -0.01120001, -0.01620001])),
  (20, array([-0.00829995, -0.0072    , -0.01209998, -0.00919998])),
  (30,
   array([-0.00709999, -0.00690001, -0.01059997, -0.0079    , -0.00709999])),
  (40,
   array([-0.00489998, -0.0108    , -0.00550002, -0.01060003, -0.00730002])),
  (50,
   array([-0.00119996, -0.00470001, -0.00830001, -0.0072    , -0.0043    ])),
  (60,
   array([-0.00380003, -0.00459999, -0.00749999, -0.00579995, -0.00730002])),
  (70,
   array([-0.00599998, -0.00199997, -0.0018    , -0.0054    , -0.00370002])),
  (80,
   array([-0.00440001, -0.00150001, -0.00379997, -0.0018    , -0.00330001])),
  (90,
   array([-0.00490004, -0.00260001, -0.00620002, -0.00489998, -0.00089997])),
  (100,
   array([-0.00569999, -0.00400001, -0.00130004, -0.00419998,  0.00130004])),
  (110,
   array([ 0.00220001, -0.00049996,  0.00190002,  0.00340003, -0.00200003])),
  (120,
   array([-0.00100005, -0.00060004, -0.00049996, -0.00230002,  0.00049996])),
  (130,
   array([-0.0007    ,  0.00279999,  0.00280005,  0.00190002,  0.00039995])),
  (140,
   array([-0.00389999,  0.00330001,  0.00200003,  0.00150001,  0.00299996])),
  (150,
   array([ 0.00480002,  0.00159997, -0.00099999,  0.00059998,  0.00230002])),
  (160,
   array([-0.00099999,  0.0011    ,  0.00390005, -0.0025    ,  0.00450003])),
  (170, array([0.00450003, 0.00779998, 0.00209999, 0.00099999, 0.00340003])),
  (178,
   array([-0.00170004,  0.00580001,  0.00239998,  0.00629997, -0.00010002])),
  (180,
   array([-0.00130004,  0.00219995,  0.00260001, -0.00059998,  0.00520003])),
  (181,
   array([-0.00690001, -0.01340002, -0.01660001, -0.00949997, -0.0147    ])),
  (182,
   array([-0.02060002, -0.0212    , -0.0176    , -0.0248    , -0.01750004]))],
 [(0,
   array([-0.60360003, -0.55649999, -0.62630001, -0.6293    , -0.56849998])),
  (1, array([-0.09909999, -0.08920002, -0.10820001, -0.11880004])),
  (2,
   array([-0.02969998, -0.03789997, -0.03720003, -0.03659999, -0.0309    ])),
  (4,
   array([-0.02060002, -0.02539998, -0.01880002, -0.02639997, -0.02079999])),
  (10,
   array([-0.01359999, -0.01599997, -0.01319999, -0.01519996, -0.01729995])),
  (20, array([-0.0108    , -0.00739998, -0.01159996, -0.01229995])),
  (30,
   array([-0.01059997, -0.00980002, -0.01410002, -0.00840002, -0.00879997])),
  (40,
   array([-0.00819999, -0.0151    , -0.00990003, -0.00999999, -0.01069999])),
  (50,
   array([-0.00239998, -0.00560004, -0.01099998, -0.01090002, -0.00779998])),
  (60,
   array([-0.00750005, -0.00669998, -0.00669998, -0.00919998, -0.00929999])),
  (70,
   array([-0.00589997, -0.00269997, -0.00310004, -0.00960004, -0.00660002])),
  (80,
   array([-0.00830001, -0.00559998, -0.00949997, -0.00440001, -0.00560004])),
  (90,
   array([-0.00600004, -0.00269997, -0.00889999, -0.00649995, -0.00369996])),
  (100,
   array([-0.00690001, -0.00419998, -0.00150001, -0.00389999,  0.00160003])),
  (110,
   array([-0.00120002, -0.00349998,  0.00040001,  0.00130004, -0.00120002])),
  (120,
   array([-0.00330001, -0.00190002, -0.00219995, -0.00410002, -0.0036    ])),
  (130,
   array([-0.00230002,  0.00190002,  0.00380003, -0.0011    ,  0.00089997])),
  (140,
   array([-0.00489998,  0.00190002, -0.00209999,  0.00150001,  0.00169998])),
  (150,
   array([ 0.0036    , -0.00170004,  0.0007    ,  0.00029999,  0.0036    ])),
  (160,
   array([-0.00209999,  0.00169998,  0.0029    , -0.00379997,  0.00319999])),
  (170, array([0.0036    , 0.00590003, 0.00060004, 0.00189996, 0.00050002])),
  (178, array([0.0007    , 0.00590003, 0.00220001, 0.00459999, 0.0011    ])),
  (180,
   array([-0.00190002, -0.00310004,  0.0036    , -0.00169998,  0.00330001])),
  (181,
   array([-0.00639999, -0.0176    , -0.0169    , -0.01179999, -0.01370001])),
  (182,
   array([-0.0237    , -0.02560002, -0.02490002, -0.0255    , -0.0266    ]))]]

In [ ]:
import utils
dirs = '''gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_71108
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_70717
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_70326
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_69545
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_67201
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_63295
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_59389
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_55483
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_51576
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_47670
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_43764
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_39858
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_35951
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_32045
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_28139
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_24233
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_20326
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_16420
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_12514
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_8608
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_4701
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_1576
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_795
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_404
gs://REDACTED/results/resnet20/global_magnitude_20/v2/lottery/prune_14'''.split()

In [ ]:
experiments = utils.experiments_of_directories(dirs)
